In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
#export
from exp.nb_05b import *
torch.set_num_threads(2)

# ConvNet

In [4]:
x_train, y_train, x_valid, y_valid = get_data()

In [5]:
#export
def normalize_to(train, valid): 
    m, s = train.mean(), train.std()
    return normalize(train, m, s), normalize(valid, m, s)

In [6]:
x_train, x_valid = normalize_to(x_train, x_valid)
train_ds, valid_ds = Dataset(x_train, y_train), Dataset(x_valid, y_valid)

In [7]:
x_train.mean(), x_train.std()

(tensor(0.0001), tensor(1.))

In [8]:
nh, bs = 50, 512
c = y_train.max().item()+1
loss_func = F.cross_entropy
data = DataBunch(*get_dls(train_ds, valid_ds, bs), c)

We define a `Lambda` layer to reshape the data for input to the model. 
It is important to know that the layer will not save parameters of a lamba function. the function must be defined.

In [9]:
#export
class Lambda(nn.Module):
    def __init__(self, func):
        super().__init__()
        self.func = func
    def forward(self, x): return self.func(x)
    
def flatten(x): return x.view(x.shape[0],-1)

This one does the other conversion: takes images of size bs * 784 and converts it to bs * 28 * 28

In [10]:
def mnist_resize(x): return x.reshape(-1, 1, 28, 28)

We can now get a simple CNN

In [11]:
nn.Conv2d??

Init signature:
nn.Conv2d(
    in_channels,
    out_channels,
    kernel_size,
    stride=1,
    padding=0,
    dilation=1,
    groups=1,
    bias=True,
    padding_mode='zeros',
)
Source:        
class Conv2d(_ConvNd):
    r"""Applies a 2D convolution over an input signal composed of several input
    planes.

    In the simplest case, the output value of the layer with input size
    :math:`(N, C_{\text{in}}, H, W)` and output :math:`(N, C_{\text{out}}, H_{\text{out}}, W_{\text{out}})`
    can be precisely described as:

    .. math::
        \text{out}(N_i, C_{\text{out}_j}) = \text{bias}(C_{\text{out}_j}) +
        \sum_{k = 0}^{C_{\text{in}} - 1} \text{weight}(C_{\text{out}_j}, k) \star \text{input}(N_i, k)


    where :math:`\star` is the valid 2D `cross-correlation`_ operator,
    :math:`N` is a batch size, :math:`C` denotes a number of channels,
    :math:`H` is a height of input planes in pixels, and :math:`W` is
    width in pixels.

    * :attr:`stride` controls the stride f

In [18]:
def get_cnn_model(data):
    return nn.Sequential(
        Lambda(mnist_resize),
        nn.Conv2d( 1, 8, 5, padding=2, stride=2), nn.ReLU(), #14
        nn.Conv2d( 8,16, 3, padding=1, stride=2), nn.ReLU(), #7
        nn.Conv2d(16,32, 3, padding=1, stride=2), nn.ReLU(), #4
        nn.Conv2d(32,32, 3, padding=1, stride=2), nn.ReLU(), #2
        nn.AdaptiveAvgPool2d(1),
        Lambda(flatten),
        nn.Linear(32, data.c)
    )

In [19]:
model = get_cnn_model(data)

In [20]:
cbfs = [Recorder, partial(AvgStatsCallback,accuracy)]

In [21]:
opt = optim.SGD(model.parameters(), lr=0.4)
learn = Learner(model, opt, loss_func, data)


In [22]:
run = Runner(cb_funcs=cbfs)

In [23]:
%time run.fit(1, learn)

train: [2.10172765625, tensor(0.2614)]
valid: [1.7630951171875, tensor(0.3957)]
CPU times: user 5.12 s, sys: 685 ms, total: 5.81 s
Wall time: 5.81 s


# CUDA

This took a long time to run, now let's use cuda 

In [25]:
# Somewhat more flexible way
device = torch.device('cuda', 0)

In [27]:
class CudaCallback(Callback):
    def __init__(self, device): self.device = device
    def begin_fit(self): self.model.to(self.device)
    def begin_batch(self): self.run.xb, self.run.yb = self.xb.to(self.device), self.yb.to(self.device)

In [29]:
# Somewhat less flexible but quite convenient
torch.cuda.set_device(device)

AttributeError: module 'torch._C' has no attribute '_cuda_setDevice'

In [30]:
#export
class CudaCallback(Callback):
    def begin_fit(self): self.model.cuda()
    def begin_batch(self): self.run.xb, self.run.yb = self.xb.cuda(), self.yb.cuda()

In [31]:
cbfs.append(CudaCallback)

In [32]:
model = get_cnn_model(data)

In [33]:
opt = optim.SGD(model.parameters(), lr=0.4)
learn = Learner(model, opt, loss_func, data)
run = Runner(cb_funcs=cbfs)

In [34]:
%time run.fit(3, learn)

AssertionError: Torch not compiled with CUDA enabled